<center><img src="img/skillfactorylogo.png"></center>

<h1><center>Курс "Практический Machine Learning"</center></h1>
<h3><center>Шестаков Андрей</center></h3>
<hr>
<h2><center>Повторение метрик качества</center></h2>
<h2><center>Метрические методы, KNN</center></h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

from ipywidgets import interact, IntSlider, FloatSlider

# Небольшое повторение

## Обобщающая способность алгоритма

* Можно построить алгоритм с идеальным качеством на обучающей выборке
* Profit???
* Nope...
* Важно понимать, насколько хорошо работает алгоритм на объектах, которые он ранее не видел


* Недообучение (underfitting) – модель сильно упростила искомую зависимость
* Переобучение (overfitting) – модель подогналась даже под шум в данных

<center><img src='img/overfit.png' width=900></center>

## Оценка обобщающей способности

<center><img src='img/cv.png' width=900></center>

# Меры качества классификации

## Меры качества классификации

* Как правило, классификаторы выдают не просто предсказанную метку класса, но и степень уверенности в ней
* Основные меры качества
    * Accuracy
    * Precision, Recall, F-measure
    * ROC-AUC, PR-AUC Gini-index, Model-lift
    * Log-loss

## Матрица перемешивания

Матрица перемешивания $M=\{m_{ij}\}_{i,j=1}^{C}$ показывает количество объектов класса $с_{i}$, которые были отнесены классификатором к классу $с_{j}$.

<center><img src='img/confusion.jpg'><center/>

## Матрица перемешивания (2 класса)


<center><img src='img/binary_conf.png'></center>
* TP (true positive) - Верное предсказание (+1)
* FP (false positive) - Ошибка первого рода (ложная тревога)
* FN (false negative) - Ошибка второго рода (пропуск цели)
* TN (true negative) - Верное предсказание (-1)
* Pos (Neg) - Общее количество +1 и -1

## Меры на основе матрицы перемешивания

<center><img src='img/binary_conf.png'></center>


* $ \text{accuracy} = \frac{TP + TN}{Pos+Neg}$
* $ \text{error rate} = 1 -\text{accuracy}$
* $ \text{recall} =\frac{TP}{TP + FN} = \frac{TP}{Pos}$ - (полнота)
* $ \text{precision} =\frac{TP}{TP + FP}$ - (точность)
* $ \text{F}_\beta \text{-score} = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}}$
    * почему не среднее или максимум?


* Можно ли посчитать эти меры для многоклассовой классификации?

In [ ]:
from mpl_toolkits.mplot3d import Axes3D


def demo_fscore(beta=1):
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    prec = np.linspace(0, 1, 100)
    rec = np.linspace(0, 1, 100)
    
    P, R = np.meshgrid(prec, rec)
    
    F = (1 + beta**2) * (P*R)/((beta**2 * P) + R)
    
    surf = ax.plot_surface(P, R, F, cmap=plt.cm.coolwarm, alpha=0.3,
                               linewidth=0, antialiased=False)

    ax.set_zlim(-0.01, 1.01)
    
    ax.view_init(30,200)
    ax.set_xlabel('precision')
    ax.set_ylabel('recall')
    ax.set_zlabel('F-score')
    ax.set_title('beta = %.2f'%beta)

    fig.colorbar(surf)
    plt.show()
    
   

In [ ]:
fig = interact(demo_fscore, beta=FloatSlider(min=0.1, max=5, step=0.3, value=1))

## Меры качества на основе уверенности классификатора

## ROC кривая

* Выбор порога классификации - отдельная большая задача
* Можно ли как-то обойтись без него и сравнить неколько моделей?


* ROC кривая - показывает зависимость между TPR (верным предсказанием) и FPR (ложным срабатыванием)
<center><img src='img/roc.png' width=400></center>


## ROC кривая

* Классификатор $a(x)$ возвращает степень принадлежности к классу "+1" - score(x).
* Упорядочим объекты по убыванию score(x)
* Смотрим сверху вниз
    * Если объект принадлежит классу "+1" - сдвиг вверх на $1/Pos$
    * Если объект принадлежит классу "-1" - сдвиг вправо на $1/Neg$
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e"><img src='img/roc1.png'></th>
    <th class="tg-031e"><img src='img/roc2.png'></th>
  </tr>
</table>



## Как сравнивать ROC-кривые?
<center><img src='img/roc.png'></center>

## ROC-AUC
* Площадь под ROC кривой

* AUC$\in[0,1]$
    * AUC = 0.5 - случайный классификатора
    * AUC = 1 - идеальный классификатор

## Precision-Recall Кривая

* Строится аналогичным образом, но по осям Precision и Recall для разных порогов

<center><img src='http://scikit-learn.org/stable/_images/sphx_glr_plot_precision_recall_001.png'></center>

## Model Lift

* Редко используется при соревнованиях по ML и DA
* Очень любят и понимают заказчики


* Вновь упорядочим объекты по убыванию score(x)
* Разобьем на процентили
* Посчитаем отношение доли +1 в процентиле к доле +1 по всей выборке
<center><img src='img/lift.png' width=700></center>


# Метод Ближайших соседей

## Гипотеза компактности

Метрические методы классификации - одни из самых простых моделей. Они основаны на гипотезе компактности - близким объектам соответствуют близкие ответы.


<center><img src='http://ichef-1.bbci.co.uk/news/624/cpsprodpb/13ED9/production/_87552618_fourupcomp.jpg' width=500></center>

* Объекты: Семьи, индивиды
* Признаки: Адрес проживания, почтовый индекс, популярный супермаркет... $\rightarrow$ координаты `(lat, lon)`
* Предскание: Раса (классификация)

<center><img src='img\california_house.png' width=500></center>

* Объекты: Недвижимость
* Признаки: Адрес дома... $\rightarrow$ координаты `(lat, lon)`
* Предсказание: Стоимость дома (регрессия)

<center><img src='img\dna_seq.png' width=900></center>
* Объекты: Строки
* Признаки: ??
* Предсказание: Функциональность гена (класс)

<hr>
<center><img src='img\text_classify.png' width=600></center>
* Объекты: Тексты статей, постов блогов
* Признаки: Частоты слов
* Предсказание: Категория текста (класс)

# Меры близости

* Как определить похожие объекты?

* Необходимо ввести функцию расстояния (не обязательно метрику)

### Самые популярные

$$ d(a, b) = \sum\limits_{i=1}^{D}(a_i - b_i)^2 \text{: euclidean distance} $$

$$ d(a, b) = \sum\limits_{i=1}^{D}|a_i - b_i| \text{: manhattan distance} $$

$$ d(a, b) = 1 - \frac{\langle a,b \rangle}{||a||_2\cdot||b||_2} \text{: cosine distance} $$



### Близость на строках
* Расстояние Левинштейна
Количество вставок, замен и удалений, которое необходимо сделать, чтобы получить из строки $S_1$ строку $S_2$

<center><img src='img\levinstein_dist.png' width=400></center>

### Близость на множествах
* Пусть объект описываеться набором категорий, слов, тегов
    * Клиент a: {Картофель фри, биг-мак, кофе, маффин}
    * Клиент b: {Картофель фри, сырный соус, чизбургер, кофе, пирожок}
* Расстояние Жаккара - Jaccard distance:
    * $$d(a,b) = 1 - \frac{|a \cap b|}{|a \cup b|}$$
    * $$d(a,b) = 1 - \frac{2}{7} = \frac{5}{7} $$
* При правильном представлении данных, можно считать и косинус

# Метод k ближайших соседей

Вход: Обучающая выборка $X=(x_i,y_i)$, мера близости $d(\cdot, \cdot)$ и объект $\tilde{x}$<br/>

Найти $k$ ближайших объекта в $X$ c помощью $d(\tilde{x},\cdot)$ 
* (классификация) вернуть наиболее частую метку класса
* (вероятность) вернуть долю объектов каждого из классов среди ближайших соседей


<center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/220px-KnnClassification.svg.png' width=400></center>

#### Начинаем щупать kNN и sklearn!

Попробуем kNN на игрушечном датасете

In [ ]:
from sklearn.datasets import make_moons

plt.style.use('ggplot')

%matplotlib inline

In [ ]:
X, y = make_moons(noise=0.3, random_state=123)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

knn = KNeighborsClassifier(n_neighbors=1, weights='uniform', metric='euclidean')
knn.fit(X, y)

In [ ]:
y_hat = knn.predict(X)

In [ ]:
y_hat_proba = knn.predict_proba(X)
y_hat_proba.shape

In [ ]:
x_range = np.linspace(X.min(), X.max(), 100)
# ОДЗ значений признаков

xx1, xx2 = np.meshgrid(x_range, x_range)
# всевозможные попарные значения признаков

Y = knn.predict_proba(np.c_[xx1.ravel(), xx2.ravel()])[:,1]
Y = Y.reshape(xx1.shape)

plt.contourf(xx1, xx2, Y, alpha=0.3)
plt.scatter(X[:,0], X[:,1],c=y)

Проблема - никак не учитывается важность соседей в зависимости от их близости!

## Взвешенный KNN

* (классификация) вернуть наиболее частую метку класса c учетом веса
* (вероятноть) вернуть долю каждого класса с учетом весов
$$\hat{y} = \arg \max\limits_{y \in \{-1, 1\}}\sum\limits_{j=1}^k w_{(j)} [y_{(j)} == y] $$

### Варианты весов
* $w_{(j)} = \frac{k - j + 1}{k}$
* $w_{(j)} = 1/d(\tilde{x},x_{(j)})$
* $w_{(j)} = K(\frac{d(\tilde{x},x_{(j)})}{h}) $
    * $K$ - ядро, $h$ - ширина окна


### Ядра
* $K(d, h) \propto \exp(- \frac{d^2}{2h^2})$ - gaussian kernel
* $K(d, h) \propto 1 if x < d$ - tophat kernel
* $K(d, h) \propto 1 - \frac{d^2}{h^2}$ - epanechnikov kernel
* $K(d, h) \propto \exp(-d/h)$ - exponential kernel
* $K(d, h) \propto 1 - d/h if d < h$ - linear kernel
* $K(d, h) \propto \cos(\frac{\pi d}{2h}) if x < h$ - linear kernel

<center><img src='http://scikit-learn.org/stable/_images/sphx_glr_plot_kde_1d_0021.png'></center>

In [ ]:
def plot_knn_class_kernel(k=1, prob=False, use_dist=False):
        
    plt.scatter(X[:,0], X[:,1], c=y)
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    
    if use_dist:
        knn = KNeighborsClassifier(n_neighbors=70, metric='minkowski', p=2, weights='distance')
    else:
        knn = KNeighborsClassifier(n_neighbors=k, metric='minkowski', p=2, weights='uniform')
    knn.fit(X, y)
    
    x_range = np.linspace(X.min(), X.max(), 500)
    # ОДЗ значений признаков

    xx1, xx2 = np.meshgrid(x_range, x_range)
    # всевозможные попарные значения признаков
    if prob:
        Y = knn.predict_proba(np.c_[xx1.ravel(), xx2.ravel()])[:,1]
    else:
        Y = knn.predict(np.c_[xx1.ravel(), xx2.ravel()])
    Y = Y.reshape(xx1.shape)

    plt.contourf(xx1, xx2, Y, alpha=0.3)
    plt.scatter(X[:,0], X[:,1], c=y)
    
    plt.show()

In [ ]:
fig = interact(plot_knn_class_kernel, k=IntSlider(min=1, max=10, value=1))

# Параметры vs Гиперпараметры

При работе с моделями следует различать понятия **Параметр** и **Гипер-параметр**.

* **Параметр** - составляющая модели, которая определяется в процессе обучения (решения оптимизационной задачи)
    * Веса коэффициентов в модели линейной регрессии
* **Гиперпараметр** - составляющая модели, которая задается перед началом обучения. Может регулировать некоторые свойства модели (скорость оптимизации) или ее сложность
    * Коэффициент регуляризации в линейной регрессии
    
Как дела обстоят у kNN?


#### Гиперпараметры kNN

* Число соседей
* Функция расстояния
* Ядро, ширина окна

# Данные с оттоком клиентов

Загрузите данные об оттоке клиентов через pandas.

In [ ]:
df_churn = pd.read_csv('data/churn.csv')

In [ ]:
df_churn.head().T

In [ ]:
print(df_churn.shape)

Это небольшие данные, но у них есть большие проблемы:

#### Проблема 1: Признаки разных шкал

In [ ]:
df_churn.describe(include='all').T

#### Проблема 2: Региональные признак
Мы можем использовать стандартную схему с OneHotEncoding:

для каждой категории создается бинарная маска-столбец, в которой стоит `1` если значение признака равно значению соответствующей категории.

Например, для признака "порт погрзки" создается бинарная макска из трех стобцов для каждого из портов: С, Q, S<br/>
Если пассажир заходил на кораблю в порту Southampton, то его признаковое описание будет соответственно: `[0, 0, 1]`


В общем виде это выглядит как-то так:
<img src='https://i.imgur.com/mtimFxh.png'>

In [ ]:
df_churn.loc[:, 'State'].nunique()

In [ ]:
df_churn.State.head()

Можно ли его как-то неявно включить в модель?

Почитайте [тут](https://alexanderdyakonov.wordpress.com/2016/08/03/python-%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5-%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%B8/)

#### Проблема 3: Довольно сильный перекос классов

In [ ]:
df_churn.loc[:, 'Churn?'].value_counts()

1. Подготовим данные
    1. Удалите ненужные признаки
    1. Преобразуйте категориальные и бинарные признаки
    1. Преобразуйте целевую переменную

In [ ]:
def preproc(df_init):
    df_preproc = df_init.copy()
    
    # Удалили признаки
    
    
    # Замена бинарных признаков
    
    
    # Исправление целевой переменной
    
    return df_preproc

In [ ]:
df_preproc = df_churn.pipe(preproc)

In [ ]:
X, y = df_preproc.iloc[:, :-1].values, df_preproc.iloc[:, -1].values

Будем использовать метод отложенной выборки

In [1]:
# Случайным образом делим на обучение и контроль
X_train, y_train, X_test, y_test = ...

In [ ]:
# Сделаем пайплайн
# 1) Нормализатор признаков
# 2) kNN

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
y_hat_proba = model.predict_proba(X_test, )

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_hat_proba[:, 1])

Зафиксируйте все гиперпараметры, кроме количества ближайших соседей, найдите наилучшее значение
* Пока сделаем это в цикле


In [ ]:
scores_test = []
scores_train = []

for k in range(1, 15):
    # Your Code Here    
    

In [ ]:
plt.plot(scores_test, label='test')
plt.plot(scores_train, label='train')
plt.legend()

# Дополнительная информация:


* [Data Mining and Analysis: Fundamental Concepts and Algorithms](https://repo.palkeo.com/algo/information-retrieval/Data%20mining%20and%20analysis.pdf) Ch 22 
* [Effective Pandas](https://github.com/TomAugspurger/effective-pandas)
* [Про структуры для поиска соседей](https://jakevdp.github.io/blog/2013/04/29/benchmarking-nearest-neighbor-searches-in-python/)